In [1]:
import pandas as pd
import json
import numpy as np

In [2]:
df = pd.read_csv('PA_test_task2_JD.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2159896 entries, 0 to 2159895
Data columns (total 4 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   user_id           object
 1   event_time        object
 2   event_type        object
 3   event_properties  object
dtypes: object(4)
memory usage: 65.9+ MB


Спочатку я хочу поділити дані, аби було зручніше робити аналітику. 

я вже проглянув попередньо датасет і бачу, що дві групи виділяються на основі 

experiment - started == {"variation_id":"0"} 

experiment - started == {"variation_id":"1"} 

In [4]:
exp = df.loc[df["event_type"] == "experiment - started", ["user_id", "event_time", "event_properties"]].copy()

#парсинг JSON
exp["event_properties"] = exp["event_properties"].fillna("{}")

def safe_json_loads(x):
    if isinstance(x, dict):
        return x
    if not isinstance(x, str) or x.strip() == "":
        return {}
    try:
        return json.loads(x)
    except Exception:
        return {}

props = exp["event_properties"].apply(safe_json_loads)
exp["variation_id"] = props.apply(lambda d: str(d.get("variation_id")) if d.get("variation_id") is not None else None)

exp["event_time"] = pd.to_datetime(exp["event_time"], utc=True, errors="coerce")
exp = exp.dropna(subset=["user_id", "event_time", "variation_id"])

#беремо перще призначення для кожного юзера
assign = (
    exp.sort_values("event_time")
       .groupby("user_id", as_index=False)
       .first()[["user_id", "variation_id"]]
)

#мапим в test/control
assign["ab_group"] = assign["variation_id"].map({"0": "control", "1": "test"})


df = df.merge(assign[["user_id", "ab_group"]], on="user_id", how="left")

#check:

#івенти
print(df["ab_group"].value_counts(dropna=False))

#спліт по користувачах
print(df.drop_duplicates("user_id")["ab_group"].value_counts())
print(df.drop_duplicates("user_id")["ab_group"].value_counts(normalize=True))

ab_group
control    1277292
test        881437
NaN           1167
Name: count, dtype: int64
ab_group
control    38342
test       16563
Name: count, dtype: int64
ab_group
control    0.698333
test       0.301667
Name: proportion, dtype: float64


тут є NaN = 1167, але це досить мала частина від даних(близько 0.05%), тому ми можемо це викинути. 


In [5]:
df_ab = df.dropna(subset=["ab_group"]).copy()

тепер зручно бачити статус користувачів

Базовою нуль-гіпотезою я виділив: Додавання туторіалу не збільшило рівень задоволеності відповідями (к-сть лайків відповіді). Водночас я би хотів оцінити інші фактори і, можливо, створити кращу гіпотезу. По-перше, варто подивитись на к-сть користувачів, які пройшли туторіал, також яка к-сть користувачів із тестової вибірки залишає фідбек загалом vs така к-сть з контрольної вибірки. Також варто оцінити adoption rate самого туторіалу(тобто чи хтось клікає на перше вікно чи проходить загалом). 

Загалом, мій хід думок полягав в тому, на що може вплинути наявність ви відсутність туторіалу на платформі(особливо з точки зору людини, яка майже ніколи не проходить туторіали, а витрачає білььше часу, аби розібратись самому.)

Як на мене, туторіал може впливати на задоволеність роботою тулзи(або можна сказати, що задоволеність результатом роботи), адже якщо користувач яркза розуміє логіку фічі, то краще можу користуватись і вдповідно не мати хибних очікувань, а задоволеність якраз вибудовується якщо очікування і реальність збігаються, retention(вже як рещультат задоволеності). 

В контексті оцінки цих факторів я думав над тим, чи можлива більша задоволеність у тестовій вибірці не може бути пов'язана із тим, що користувачі, які пройшли туторіал будуть більш вмотивовані і відповідно більш задоволені роботою тулки, тому в гіпотезі варто оцінювати результати усієї вибірки, а не лише ту частку користувачів, які пройшли туторіал. 

Тому в підсумку:

H0: Додавання інтерактивного туторіалу не підвищує рівень задоволеності результатами роботи тулів.


H1: Додавання інтерактивного туторіалу підвищує рівень задоволеності результатами роботи тулів.

In [6]:
# мій перший інстинкт - це було подивитись на абсолютну к-сть лайків у двох різних групах та знайти співвідношення

like_dislike_counts = (
    df_ab[df_ab["event_type"].isin(["response - click like", "response - click dislike"])]
    .groupby(["ab_group", "event_type"])
    .size()
    .unstack(fill_value=0)
)

like_dislike_counts["total_reactions"] = (
    like_dislike_counts["response - click like"] +
    like_dislike_counts["response - click dislike"]
)

like_dislike_counts["like_share"] = (
    like_dislike_counts["response - click like"] / like_dislike_counts["total_reactions"]
)

like_dislike_counts["dislike_share"] = (
    like_dislike_counts["response - click dislike"] / like_dislike_counts["total_reactions"]
)

like_dislike_counts

event_type,response - click dislike,response - click like,total_reactions,like_share,dislike_share
ab_group,,,,,
control,1817,2751,4568,0.602233,0.397767
test,683,1198,1881,0.636895,0.363105


але мені здається, що це можуть бути трішки misleading результати, адже тут ми рахуємо співвідношення між абсолютною к-стю лайків та дизлайків у кожній групі і це значить, що надто активні позитивні/негативні юзери можуть змістити наші значення, тому я хочу спробувати агрегувати ці значення на базі кожного юзера і подивитись, який буде результат тоді. 

In [7]:
reactions = df_ab[df_ab["event_type"].isin(["response - click like", "response - click dislike"])].copy()
reactions["cnt"] = 1

user_reactions = (
    reactions.pivot_table(
        index=["user_id", "ab_group"],
        columns="event_type",
        values="cnt",
        aggfunc="sum",
        fill_value=0
    )
    .reset_index()
    .rename(columns={
        "response - click like": "likes",
        "response - click dislike": "dislikes"
    })
)

user_reactions["total"] = user_reactions["likes"] + user_reactions["dislikes"]
user_reactions["satisfaction_rate"] = user_reactions["likes"] / user_reactions["total"]

group_satisfaction_mean = user_reactions.groupby("ab_group")["satisfaction_rate"].mean()
group_satisfaction_mean

ab_group
control    0.630423
test       0.652679
Name: satisfaction_rate, dtype: float64

Ми вже бачимо певні результати, але тепер треба красиво це виконати і створити більш формальний A/B тест. 

In [8]:
primary_summary = (
    user_reactions.groupby("ab_group")["satisfaction_rate"]
    .agg(
        users_reacted="count",
        mean="mean",
        median="median",
        p25=lambda x: x.quantile(0.25),
        p75=lambda x: x.quantile(0.75),
    )
)

primary_summary

,users_reacted,mean,median,p25,p75
ab_group,,,,,
control,3498,0.630423,1.0,0.0,1.0
test,1476,0.652679,1.0,0.0,1.0


In [9]:
control_mean = primary_summary.loc["control", "mean"]
test_mean = primary_summary.loc["test", "mean"]

uplift_abs = test_mean - control_mean
uplift_rel = uplift_abs / control_mean

print(f"Control mean satisfaction: {control_mean:.4f}")
print(f"Test mean satisfaction:    {test_mean:.4f}")
print(f"Absolute uplift:           {uplift_abs:.4f} ({uplift_abs*100:.2f} pp)")
print(f"Relative uplift:           {uplift_rel*100:.2f}%")


Control mean satisfaction: 0.6304
Test mean satisfaction:    0.6527
Absolute uplift:           0.0223 (2.23 pp)
Relative uplift:           3.53%


Отож, результат ідеї порахувати метрику задоволеності для кожного юзера:

- Test mean satisfaction > Control mean satisfaction
- uplift = 2.23 (та Relative uplift: 3.53%)

Це значить, що в тестовій групі користувачі (які залишають реакції) частіше оцінюють результат позитивно, ніж у контрольній.

In [10]:
#хто користувався тулками
tool_users = (
    df_ab[df_ab["event_type"] == "tools - send message"][["user_id", "ab_group"]]
    .drop_duplicates()
)

#користувачі, які задлишили реакцію
react_users = (
    df_ab[df_ab["event_type"].isin(["response - click like", "response - click dislike"])][["user_id", "ab_group"]]
    .drop_duplicates()
)

feedback_rate = (
    tool_users.groupby("ab_group")["user_id"].nunique()
    .rename("tool_users")
    .to_frame()
    .join(react_users.groupby("ab_group")["user_id"].nunique().rename("react_users"))
)

feedback_rate["feedback_rate"] = feedback_rate["react_users"] / feedback_rate["tool_users"]
feedback_rate

,tool_users,react_users,feedback_rate
ab_group,,,
control,37305,3498,0.093768
test,16123,1476,0.091546


Окремо я порахував feedback_rate, щоб зрозуміти чи туторіал впливає на поведінку “залишати фідбек”, а не тільки на satisfaction.

primary metric я рахую тільки серед тих, хто реагує. Якщо feedback_rate різний між групами, то можливий bias.

Далі я порівняю feedback_rate між control і test, щоб підтвердити, що порівнювати satisfaction правильно.

In [11]:
test_users_viewed_tool = (
    df_ab[
        (df_ab["ab_group"] == "test") &
        (df_ab["event_type"].str.contains(r"view.*tool", case=False, na=False))
    ][["user_id"]]
    .drop_duplicates()
)

test_users_tut_started = (
    df_ab[(df_ab["ab_group"] == "test") & (df_ab["event_type"] == "tutorial - started")][["user_id"]]
    .drop_duplicates()
)

test_users_tut_finished = (
    df_ab[(df_ab["ab_group"] == "test") & (df_ab["event_type"] == "tutorial - finished")][["user_id"]]
    .drop_duplicates()
)

n_viewed = len(test_users_viewed_tool)
n_started = len(test_users_tut_started)
n_finished = len(test_users_tut_finished)

tutorial_adoption = pd.DataFrame([{
    "test_users_viewed_tool": n_viewed,
    "test_users_tut_started": n_started,
    "test_users_tut_finished": n_finished,
    "tutorial_start_rate": (n_started / n_viewed) if n_viewed else np.nan,
    "tutorial_finish_rate_among_started": (n_finished / n_started) if n_started else np.nan,
    "tutorial_finish_rate_among_viewed": (n_finished / n_viewed) if n_viewed else np.nan,
}])

tutorial_adoption

,test_users_viewed_tool,test_users_tut_started,test_users_tut_finished,tutorial_start_rate,tutorial_finish_rate_among_started,tutorial_finish_rate_among_viewed
0,16315,13334,12991,0.817285,0.974276,0.796261


Також я хотів зрозуміти чи фіча реально використовується.

Тому в adoption Test я дивився на tutorial_start_rate та tutorial_finish_rate

Якщо adoption низький, то навіть корисний туторіал може показати слабкий ефект.

In [12]:
#аctivation(user-level): view tool -> send message

view_users = (
    df_ab[
        df_ab["event_type"].str.contains(r"view.*tool", case=False, na=False)
    ][["user_id", "ab_group"]]
    .drop_duplicates()
)

send_users = (
    df_ab[df_ab["event_type"] == "tools - send message"][["user_id", "ab_group"]]
    .drop_duplicates()
)

activation = (
    view_users.groupby("ab_group")["user_id"].nunique()
    .rename("view_users")
    .to_frame()
    .join(send_users.groupby("ab_group")["user_id"].nunique().rename("send_users"))
)

activation["activation_rate"] = activation["send_users"] / activation["view_users"]
activation

,view_users,send_users,activation_rate
ab_group,,,
control,37720,37305,0.988998
test,16315,16123,0.988232


Туторіал має допомогти користувачам краще зрозуміти функції тулів, додатково оцінив activation:

Ця метрика показує, чи допоміг туторіал користувачам перейти від перегляду тули до реального використання (початку генерації). Бачимо, що рещультати практично ідентичні для обох вибірок, тому можемо сказати, що туторіал майже не вплинува на активацію.

In [13]:
def bootstrap_diff_mean(a, b, n_boot=10000, seed=42):
    rng = np.random.default_rng(seed)
    a = np.asarray(a); b = np.asarray(b)
    obs = b.mean() - a.mean()

    diffs = np.empty(n_boot)
    for i in range(n_boot):
        diffs[i] = rng.choice(b, size=len(b), replace=True).mean() - rng.choice(a, size=len(a), replace=True).mean()

    ci_low, ci_high = np.quantile(diffs, [0.025, 0.975])
    p_one_sided = (diffs <= 0).mean()
    return obs, (ci_low, ci_high), p_one_sided

control = user_reactions[user_reactions["ab_group"] == "control"]["satisfaction_rate"].values
test = user_reactions[user_reactions["ab_group"] == "test"]["satisfaction_rate"].values

obs_diff, ci, p_one_sided = bootstrap_diff_mean(control, test)

print("Primary metric: mean user-level satisfaction_rate among reacting users")
print(f"Diff (Test - Control): {obs_diff:.4f}")
print(f"95% bootstrap CI: [{ci[0]:.4f}, {ci[1]:.4f}]")
print(f"One-sided p-value (H1: Test > Control): {p_one_sided:.4f}")

Primary metric: mean user-level satisfaction_rate among reacting users
Diff (Test - Control): 0.0223
95% bootstrap CI: [-0.0058, 0.0504]
One-sided p-value (H1: Test > Control): 0.0601



Я хотів перевірити, чи uplift не є випадковим шумом, томв використав bootstrap тест.

Результати bootstrap:

Diff (Test − Control) = +0.0223, тобто у тесті satisfaction вища на +2.23 pp (позитивний напрям)

95% CI = [-0.0058; 0.0504], інтервал включає 0, тобто ми не можемо виключити, що реальний ефект = 0 або навіть трохи негативний

p-value (one-sided) = 0.0601, це  >0.05, тому на рівні значущості 5% ми не відхиляємо H0

Загалом, у Test satisfaction вище на 2.23 pp., але ефект не досяг статистичної значущості (p=0.0601), а 95% CI перетинає 0. Тому на основі цих даних нема достатньо доказів, що туторіал реально підвищує satisfaction. Також, p-value близький до порогу 0.05, тому результат можна трактувати як слабкий сигнал на користь позитивного ефекту, але доказів недостатньо, аби робити повний і  впевнений rollout.

Рекомендація:

1) Не робити 100% rollout лише на основі цього


2) оптимізувати туторіал, аби збільшити adoption

3) додати secondary metrics у decision: activation / feedback_rate


Мені не давало спокою те, що я прийняв нуль-гіпотезу, але не зміг пояснити, чому так і що у продукті могло спричинити відсутність доказів, що туторіал впливає на рівень задоволеністю відповіддю у користувача, тому я постараюсь проаналізувати глибше


Я постарався виділити потенційни причини, чому я не спостеріг доказів ефекту туторіалу(дещо з цього я вже аналізував раніше, але прописав зе раз, аби було все в одному місці і можна було легко порівняти):


1) Низька статистична потужність (power) і широка невизначеність. Primary metric рахується лише серед користувачів, які залишили реакцію, тобто реальний розмір вибірки для тесту — це `users_reacted`, а не всі юзери. Якщо таких юзерів небагато або метрика має велику дисперсію, то довірчий інтервал буде надто широким і включатиме 0, тому ми не відхиляємо H0, навіть якщо uplift позитивний.

2) Низький adoption туторіалу. Я порівнюю всіх користувачів у Test vs всіх у Control. Якщо лише невелика частка Test реально почала/закінчила туторіал, то середній ефект на всю групу Test розмивається. Тобто туторіал може бути корисним для тих, хто пройшов, але в загальному аналізі ефект буде непомітним.

3) Selection bias: satisfaction рахується лише серед “reactors”. Якщо туторіал впливає на поведінку “залишати реакцію” (тобто feedback_rate різний у групах), тоді набори користувачів в Test і Control можуть бути різними. Тоді порівняння satisfaction_rate може бути зміщеним.


4) Туторіал не змінив поведінку користування. Туторіал має допомагати користувачу перейти від перегляду тули до реального використання. Якщо activation майже не відрізняється між Test і Control, це означає, що туторіал не змінив core поведінку. Тоді ефект на satisfaction  буде малим і його важко побачити статистично.

5) Метрика шумна: Якщо користувач залишив лише 1 реакцію, то satisfaction_rate = 0 або 1. Якщо таких юзерів багато, дисперсія метрики зростає, CI розширюється, і статистично значущий результат отримати важко.

In [20]:
control_mean = float(primary_summary.loc["control", "mean"])
test_mean = float(primary_summary.loc["test", "mean"])
uplift_pp = (test_mean - control_mean) * 100

n_reacted_control = int(primary_summary.loc["control", "users_reacted"])
n_reacted_test = int(primary_summary.loc["test", "users_reacted"])

print(f"Observed uplift (Test-Control): {uplift_pp:+.2f} pp")
print(f"Reacting users (effective N): control={n_reacted_control}, test={n_reacted_test}")
print(f"Bootstrap 95% CI: [{ci[0]*100:+.2f} pp; {ci[1]*100:+.2f} pp] (crosses 0? {ci[0] <= 0 <= ci[1]})")
print(f"One-sided p-value (H1: Test > Control): {p_one_sided:.4f}")

Observed uplift (Test-Control): +2.23 pp
Reacting users (effective N): control=3498, test=1476
Bootstrap 95% CI: [-0.58 pp; +5.04 pp] (crosses 0? True)
One-sided p-value (H1: Test > Control): 0.0601


In [16]:
ta = tutorial_adoption.iloc[0]

print("Tutorial adoption (Test group):")
print(f"- Viewed tool users (test):   {int(ta['test_users_viewed_tool'])}")
print(f"- Tutorial started (test):    {int(ta['test_users_tut_started'])}")
print(f"- Tutorial finished (test):   {int(ta['test_users_tut_finished'])}")
print(f"- Start rate:                 {ta['tutorial_start_rate']:.2%}")
print(f"- Finish rate among started:  {ta['tutorial_finish_rate_among_started']:.2%}")
print(f"- Finish rate among viewed:   {ta['tutorial_finish_rate_among_viewed']:.2%}")

Tutorial adoption (Test group):
- Viewed tool users (test):   16315
- Tutorial started (test):    13334
- Tutorial finished (test):   12991
- Start rate:                 81.73%
- Finish rate among started:  97.43%
- Finish rate among viewed:   79.63%


Тут одразу видно, що adoption на високому рівні, тому можемо відкинути цю причину. 

In [21]:
tool_users = (
    df_ab[df_ab["event_type"] == "tools - send message"][["user_id", "ab_group"]]
    .drop_duplicates()
)

react_users = (
    df_ab[df_ab["event_type"].isin(["response - click like", "response - click dislike"])][["user_id"]]
    .drop_duplicates()
)

tool_users = tool_users.copy()
tool_users["reacted"] = tool_users["user_id"].isin(react_users["user_id"]).astype(int)

react_rate = tool_users.groupby("ab_group")["reacted"].mean()
react_rate_control = float(react_rate.get("control", np.nan))
react_rate_test = float(react_rate.get("test", np.nan))
diff_pp = (react_rate_test - react_rate_control) * 100

print("Feedback rate (tool users who reacted at least once):")
print(f"- Control: {react_rate_control:.2%}")
print(f"- Test:    {react_rate_test:.2%}")
print(f"- Diff:    {diff_pp:+.2f} pp")

Feedback rate (tool users who reacted at least once):
- Control: 9.38%
- Test:    9.15%
- Diff:    -0.23 pp


Тут також бачимо, що різниця мафже нульова між фідбек рейтом, тому відкидаємо цю причину.(туторіал не вплинув на поведінку залишати чи ні фідбек)

In [22]:
act_control = float(activation.loc["control", "activation_rate"])
act_test = float(activation.loc["test", "activation_rate"])
act_diff_pp = (act_test - act_control) * 100

print("Activation (view tool → send message):")
print(f"- Control: {act_control:.2%}")
print(f"- Test:    {act_test:.2%}")
print(f"- Diff:    {act_diff_pp:+.2f} pp")

Activation (view tool → send message):
- Control: 98.90%
- Test:    98.82%
- Diff:    -0.08 pp


Тут бачимо, що туторіал не вплинув на основну поведінку, а саме - користуватись тулом, але я підозрюю, що тут він і не дуже зміг змінити картину, бо контрольна вибірка і так має дуже високий рівень активації і по-суті, впирається в "стелю"

In [23]:
share_total_eq_1 = user_reactions.groupby("ab_group")["total"].apply(lambda s: (s == 1).mean())
share_total_eq_1_control = float(share_total_eq_1.get("control", np.nan))
share_total_eq_1_test = float(share_total_eq_1.get("test", np.nan))

print("Share of reacting users with exactly 1 total reaction:")
print(f"- Control: {share_total_eq_1_control:.2%}")
print(f"- Test:    {share_total_eq_1_test:.2%}")

Share of reacting users with exactly 1 total reaction:
- Control: 81.22%
- Test:    82.52%


Бачимо, що більше 80% reacting users залишили лише 1 реакцію, тому satisfaction_rate для більшості користувачів = 0 або 1. Це збільшує дисперсію метрики і знижує ймовірність отримати статистично значущий результат. Як на мене, це і є реальною причиною і це підтверджується 95% bootstrap CI(з тесту на першу причину) [-0.58; +5.04] pp, який широкий і включає 0 

Тепер спробую трішки змінити гіпотезу і подивитись, чи туторіал вплинув на рівень reported feedback про саму тулзу. 

H0: Туторіал не змінює середню оцінку feedback_v3 (1–5) серед користувачів, які залишили оцінку

H1: Туторіал змінює середню оцінку feedback_v3 (1–5) серед користувачів, які залишили оцінку


Це досить схожий тест до попереднього, тому я також буду агрегувати score для кожного користувача.

In [ ]:
df_ab[df_ab['event_type'] == 'feedback_v3 - click rate bar']

,user_id,event_time,event_type,event_properties,ab_group
7044,37514297054ce4eedeb32ef28af4d3808291a49cd2b1d2...,2025-03-30 23:27:42.853 UTC,feedback_v3 - click rate bar,"{""tool"":""AI Chat"",""value"":""1""}",control
7045,e25d3a0df0d445854cf822c3f0505a451d349354ac2472...,2025-03-29 22:12:56.955 UTC,feedback_v3 - click rate bar,"{""tool"":""AI Chat"",""value"":""1""}",control
7046,aa6b5c7860f195df8bed471816e3a7941a7e37d4a90653...,2025-04-03 05:04:06.697 UTC,feedback_v3 - click rate bar,"{""tool"":""AI Chat"",""value"":""1""}",test
7047,9b4fa342b68e2b1ab598ca7f628f13d066123eccbcc740...,2025-04-03 12:40:40.082 UTC,feedback_v3 - click rate bar,"{""tool"":""AI Chat"",""value"":""1""}",test
7048,57d7e2dae22d1edcd61079cb39e8e35d4578c007f8a407...,2025-04-06 20:25:50.249 UTC,feedback_v3 - click rate bar,"{""tool"":""AI Chat"",""value"":""1""}",control
...,...,...,...,...,...
2109457,be948794153a751e1a06248b9ef1db000417efe28b1091...,2025-03-27 20:04:46.021 UTC,feedback_v3 - click rate bar,"{""tool"":""Paraphrase Text"",""value"":""4""}",test
2109458,0360870478cf93a7dd1faf26b0ee37673b067380c83fdf...,2025-03-30 07:10:01.762 UTC,feedback_v3 - click rate bar,"{""tool"":""Paraphrase Text"",""value"":""4""}",test
2109459,79e401e6b3a8ce877d2c49ee6d5a16cd183093d1d52102...,2025-03-31 05:43:19.157 UTC,feedback_v3 - click rate bar,"{""tool"":""Paraphrase Text"",""value"":""5""}",control
2109460,6905bd2f1168b2fc92020ede16e39091bc30b4c95bc699...,2025-03-31 22:39:32.372 UTC,feedback_v3 - click rate bar,"{""tool"":""Paraphrase Text"",""value"":""5""}",test


In [ ]:
fb = df_ab[df_ab["event_type"] == "feedback_v3 - click rate bar"].copy()

fb["event_properties"] = fb["event_properties"].fillna("{}")

def safe_json_loads(x):
    if isinstance(x, dict):
        return x
    if not isinstance(x, str) or x.strip() == "":
        return {}
    try:
        return json.loads(x)
    except Exception:
        return {}

props = fb["event_properties"].apply(safe_json_loads)

fb["tool"] = props.apply(lambda d: d.get("tool"))
fb["value"] = pd.to_numeric(props.apply(lambda d: d.get("value")), errors="coerce")

fb = fb.dropna(subset=["user_id", "ab_group", "tool", "value"])
fb = fb[fb["value"].between(1, 5)]

fb[["user_id", "ab_group", "tool", "value"]].head(10)

,user_id,ab_group,tool,value
7044,37514297054ce4eedeb32ef28af4d3808291a49cd2b1d2...,control,AI Chat,1
7045,e25d3a0df0d445854cf822c3f0505a451d349354ac2472...,control,AI Chat,1
7046,aa6b5c7860f195df8bed471816e3a7941a7e37d4a90653...,test,AI Chat,1
7047,9b4fa342b68e2b1ab598ca7f628f13d066123eccbcc740...,test,AI Chat,1
7048,57d7e2dae22d1edcd61079cb39e8e35d4578c007f8a407...,control,AI Chat,1
7049,6dfc218909f6089605b74747104e127452ed0912a0ead4...,control,AI Chat,4
7050,c4aec5102909a08d9155a74ca167ca70b40f3494cdef7b...,control,AI Chat,5
7051,e089a1728b84b04b4e3fba7edaff5980abbdada61cff01...,control,AI Chat,5
7052,5ffe208a20f62c87e6ef6ac2e086127a2dcceebc19224b...,test,AI Chat,5
7053,a02f6b60b2a76d40ff457a4d1a7eeb2128c538835b532d...,control,AI Chat,5


In [ ]:
user_tool = (
    fb.groupby(["user_id", "ab_group", "tool"])["value"]
      .mean()
      .reset_index()
      .rename(columns={"value": "user_avg_rating"})
)


tool_group_user = (
    user_tool.groupby(["tool", "ab_group"])["user_avg_rating"]
      .agg(
          n_users="count",
          mean_rating="mean",
          median_rating="median"
      )
      .reset_index()
      .sort_values(["tool", "ab_group"])
)

tool_group_user

,tool,ab_group,n_users,mean_rating,median_rating
0,AI Chat,control,684,3.952494,5.00
1,AI Chat,test,274,3.935158,5.00
2,AI Content Detector,control,366,3.638889,5.00
3,AI Content Detector,test,151,3.995033,5.00
4,AI Researcher,control,65,4.156410,5.00
5,AI Researcher,test,28,4.035714,5.00
6,AI Text Humanizer,control,1788,3.926547,5.00
7,AI Text Humanizer,test,746,3.877234,5.00
8,Bildgenerator,control,1,5.000000,5.00
9,Book Writer (Chapter Format),control,1,5.000000,5.00


In [47]:
pivot_user = tool_group_user.pivot(index="tool", columns="ab_group", values=["n_users", "mean_rating"])
pivot_user.columns = [f"{a}_{b}" for a, b in pivot_user.columns]
pivot_user = pivot_user.reset_index()

pivot_user["diff_test_minus_control"] = pivot_user["mean_rating_test"] - pivot_user["mean_rating_control"]
pivot_user["rel_uplift_%"] = 100 * pivot_user["diff_test_minus_control"] / pivot_user["mean_rating_control"]

pivot_user.sort_values("diff_test_minus_control", ascending=False)


def bootstrap_diff_mean(a, b, n_boot=20000, seed=42):
    rng = np.random.default_rng(seed)
    a = np.asarray(a); b = np.asarray(b)
    obs = b.mean() - a.mean()

    diffs = np.empty(n_boot)
    for i in range(n_boot):
        diffs[i] = rng.choice(b, size=len(b), replace=True).mean() - rng.choice(a, size=len(a), replace=True).mean()

    ci_low, ci_high = np.quantile(diffs, [0.025, 0.975])
    p_two_sided = 2 * min((diffs <= 0).mean(), (diffs >= 0).mean())
    return obs, (ci_low, ci_high), float(min(1.0, p_two_sided))

rows = []
for tool, sub in user_tool.groupby("tool"):
    control = sub[sub["ab_group"] == "control"]["user_avg_rating"].values
    test = sub[sub["ab_group"] == "test"]["user_avg_rating"].values

    if len(control) < 30 or len(test) < 30:
        continue

    obs, (ci_l, ci_h), p = bootstrap_diff_mean(control, test)

    rows.append({
        "tool": tool,
        "n_users_control": len(control),
        "n_users_test": len(test),
        "mean_control": control.mean(),
        "mean_test": test.mean(),
        "diff_test_minus_control": obs,
        "ci_low": ci_l,
        "ci_high": ci_h,
        "p_value": p
    })

results_user = pd.DataFrame(rows).sort_values("p_value")
results_user

,tool,n_users_control,n_users_test,mean_control,mean_test,diff_test_minus_control,ci_low,ci_high,p_value
1,AI Content Detector,366,151,3.638889,3.995033,0.356144,0.042930,0.660846,0.0249
2,AI Text Humanizer,1788,746,3.926547,3.877234,-0.049313,-0.188618,0.086426,0.4804
3,Check Plagiarism,171,68,4.105263,4.191176,0.085913,-0.357417,0.508862,0.6910
4,Paraphrase Text,120,68,4.062500,4.094118,0.031618,-0.409559,0.461765,0.8726
0,AI Chat,684,274,3.952494,3.935158,-0.017335,-0.246885,0.207930,0.8917


В результаті цих тестів можна зробити висновок, що туторіал покращив оцінку лише AI Content Detector фічі. Для інших фіч таких доказів немає. З іншого боку, я роблю кілька тестів водночас і це збільшує ймовірність false positive, тому аби це валідувати, можна провести окремий тест. Тому я би зробив висновок, що це просто сигнал про те, що цю фічу можна більше дослідити, а не обов'язково справжній вплив.